In [1]:
from Drawer import ImageDrawer
from Find_tabletop import HexagonDetector
import cv2
from roboflow import Roboflow

In [3]:
class Project:
    def __init__(self):
        self.model = []
        self.centers = []
        self.taken_centers = []
        self.deploy_model()
        self.detector = HexagonDetector()
        self.image_drawer = ImageDrawer(640, 640, hex_size=36)

    def predict(self, image):
        self.centers = self.detector.find_hexagons(image)
        if len(self.centers) == 0:
            raise Exception("No se detectaron hexágonos")

    def draw(self, image):
        self.image_drawer.draw_image(image)
        self.image_drawer.show_image()

    def deploy_model(self):
        rf = Roboflow(api_key="0TVwyZuH7ZvyGtzoNC7B")
        project = rf.workspace().project("ankh_project")
        self.model = project.version(1).model

    def predict_and_draw(self, image):
        image = cv2.resize(image, (640, 640))
        result = self.model.predict(image, confidence=40, overlap=30).json()
        if len(result['predictions']) == 0:
            print("No se detectaron estatuillas")
        else:
            temp = []
            for i in range(0, len(result['predictions'])):
                x = result['predictions'][i]['x']
                height = result['predictions'][i]['height']
                y = result['predictions'][i]['y'] - (height//2)*0.6
                closest_center = min(self.centers, key=lambda center: ((center[0] - x) ** 2 + (center[1] - y) ** 2) ** 0.5)
                temp.append(closest_center)
                if closest_center in self.taken_centers:
                    continue
                self.taken_centers.append(closest_center)
                if result['predictions'][i]['class'] == 'amon':
                    self.image_drawer.draw_square(closest_center, size=50, color=(255, 0, 0))
                elif result['predictions'][i]['class'] == 'isis':
                    self.image_drawer.draw_hexagon_object(closest_center, size=50)
                elif result['predictions'][i]['class'] == 'amon_follower':
                    self.image_drawer.draw_square(closest_center, size=20, color=(255, 0, 0))
                elif result['predictions'][i]['class'] == 'isis_follower':
                    self.image_drawer.draw_hexagon_object(closest_center, size=20)
            for centers in self.taken_centers:
                if centers not in temp:
                    self.image_drawer.remove_object(centers,size=(50, 50))
                    self.taken_centers.remove(centers)
        self.image_drawer.show_image()

In [ ]:
Proyecto=Project()
cap = cv2.VideoCapture(0)
# Check if camera is opened successfully
if not cap.isOpened():
    print("Unable to open camera")
else:
    ret, frame = cap.read()
    if not ret:
        print("Unable to read frame from camera")
    else:
        Proyecto().predict(frame)
        Proyecto().draw(frame)
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap = cv2.VideoCapture(0)
# Check if camera is opened successfully
if not cap.isOpened():
    print("Unable to open camera")
else:
    ret, frame = cap.read()
    if not ret:
        print("Unable to read frame from camera")
    else:
        Proyecto().predict_and_draw(frame)
cap.release()
cv2.destroyAllWindows()
